In [1]:
%matplotlib inline

In [4]:
from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from vtk_rw import read_vtk
import seaborn as sns
from plotting import plot_surf_stat_map
import matplotlib
from graphs import graph_from_mesh, dijkstra, shortest_path
import gdist
import networkx as nx

In [5]:
sub='BP4T'

In [6]:
simple_v, simple_f, _ = read_vtk('/scr/ilz3/myelinconnect/groupavg/indv_space/%s/lowres_rh_d_def.vtk'%sub)

In [7]:
complex_v, complex_f, _ = read_vtk('/scr/ilz3/myelinconnect/struct/surf_rh/orig/mid_surface/%s_rh_mid.vtk'%sub)

In [8]:
mapping_ideal = np.load('/scr/ilz3/myelinconnect/all_data_on_simple_surf/seeds_ideal/%s_rh_highres2lowres_seeds.npy'%sub)

In [9]:
voronoi_seed_idx = mapping_ideal

In [19]:
def edge_len(nodes, triangles, edge):
    return np.linalg.norm(nodes[edge[0]]-nodes[edge[1]])

In [20]:
def weighted_graph_from_mesh(nodes, triangles, weight_func=edge_len):
    '''
    Creates a weighted networkx graph from a mesh
    '''
    
    import numpy as np
    import networkx as nx
    
    # initiate empty graph
    G=nx.Graph()
    
    # add node indices
    G.add_nodes_from(range(len(nodes)))
    
    # add edges
    G.add_edges_from(triangles[:,[0,1]])
    G.add_edges_from(triangles[:,[0,2]])
    G.add_edges_from(triangles[:,[1,2]])
    
    if weight_func is not None:
        for e in G.edges_iter():
            G[e[0]][e[1]]['weight']=weight_func(nodes, triangles, e)
            G[e[1]][e[0]]['weight']=weight_func(nodes, triangles, e)

    return G

In [17]:
simple_graph=graph_from_mesh(simple_v, simple_f, edge_length=True)

In [21]:
complex_weighted_graph=weighted_graph_from_mesh(complex_v, complex_f, weight_func=edge_len)

In [33]:
df = pd.DataFrame(columns=['simple e0', 'simple e1', 'simple len', 
                           'complex e0', 'complex e1', 'complex len'],
                      index=np.arange(len(simple_graph.edges())))

In [ ]:
count = 0
for e in simple_graph.edges_iter():
    
    df['simple e0'][count] = e[0]
    df['simple e1'][count] = e[1]
    df['simple len'][count] = simple_graph[e[0]][e[1]]

    df['complex e0'][count] = voronoi_seed_idx[e[0]]
    df['complex e1'][count] = voronoi_seed_idx[e[1]]
    df['complex len'][count] = nx.dijkstra_path_length(complex_weighted_graph, 
                                                       voronoi_seed_idx[e[0]], 
                                                       voronoi_seed_idx[e[1]])
    
    count += 1